In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
import PyPDF2
import os
from dotenv import load_dotenv

class AnnualReportRAG:
    def __init__(self, mistral_api_key):
        """Initialize the RAG system with Mistral API key."""
        # Initialize Mistral client for text generation
        self.mistral_client = MistralClient(api_key=mistral_api_key)
        
        # Initialize embedding model (using a free alternative to Mistral embeddings)
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        
        # Initialize vector store as None (will be created when processing document)
        self.vector_store = None
        
    def load_pdf(self, pdf_path):
        """Load and extract text from PDF file."""
        pdf_text = ""
        # Open PDF file in binary read mode
        with open(pdf_path, 'rb') as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            # Iterate through each page and extract text
            for page in pdf_reader.pages:
                pdf_text += page.extract_text()
        return pdf_text
    
    def process_document(self, text):
        """Split the document into chunks and create embeddings."""
        # Initialize text splitter with specific parameters
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,  # Number of characters per chunk
            chunk_overlap=200,  # Number of overlapping characters between chunks
            length_function=len
        )
        
        # Split text into manageable chunks
        chunks = text_splitter.split_text(text)
        
        # Create FAISS vector store from text chunks
        self.vector_store = FAISS.from_texts(
            texts=chunks,
            embedding=self.embeddings
        )
    
    def get_relevant_context(self, question, k=3):
        """Retrieve relevant context for the question."""
        if self.vector_store is None:
            raise ValueError("Please process a document first!")
            
        # Search for similar documents
        docs = self.vector_store.similarity_search(question, k=k)
        
        # Combine all relevant contexts
        context = "\n\n".join([doc.page_content for doc in docs])
        return context, docs
    
    def query(self, question):
        """Query the RAG system using Mistral AI."""
        # Get relevant context
        context, source_docs = self.get_relevant_context(question)
        
        # Create prompt with context
        system_prompt = """You are a helpful AI assistant analyzing company annual reports. 
        Use the provided context to answer questions accurately. 
        If the information cannot be found in the context, say so."""
        
        # Create messages for Mistral chat completion
        messages = [
            ChatMessage(role="system", content=system_prompt),
            ChatMessage(role="user", content=f"""Context: {context}
            
            Question: {question}
            
            Please answer the question based on the context provided.""")
        ]
        
        # Get response from Mistral
        response = self.mistral_client.chat(
            model="mistral-large-2402",
            messages=messages,
            temperature=0.1,
            max_tokens=500
        )
        
        return {
            "answer": response.messages[0].content,
            "source_documents": source_docs
        }

def main():
    # Load environment variables
    load_dotenv()
    
    # Initialize RAG system with Mistral API key
    rag = AnnualReportRAG(os.getenv("Mistral_API_KEY"))
    
    # Example usage
    pdf_text = rag.load_pdf("RIL-Integrated-Annual-Report-2022-23.pdf")
    rag.process_document(pdf_text)
    
    # Example queries
    questions = [
        "What was the company's revenue in the last fiscal year?",
        "What are the main risk factors mentioned in the report?",
        "What is the company's strategy for growth?"
    ]
    
    for question in questions:
        result = rag.query(question)
        print(f"\nQuestion: {question}")
        print(f"Answer: {result['answer']}")
        print("\nSources:")
        for doc in result['source_documents']:
            print(f"- {doc.page_content[:200]}...")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'mistralai.models.chat_completion'